In [1]:
import oracledb
import pandas as pd
from sqlalchemy import create_engine
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# connect to oracle database
connection=oracledb.connect(
     user="Profile_nv2",
     password="mbf52023flex",
     dsn="10.1.179.2:1521/appdb.mobifone5.vn",
)
# create engine
engine = create_engine('oracle+oracledb://', creator=lambda: connection)

In [2]:
import pypi_xmlrpc #pip install pypi-xmlrpc
import csv
from tqdm import tqdm
import pandas as pd

# server = 'http://localhost:8069'
# database = "Diem_Danh_Hik_odoo17_8"
# user = "doanhdai1997@gmail.com"
# password = "Baodai123!"

server = 'https://faceid.mobifone5.vn'
database = "faceID"
user = "doanhdai1997@gmail.com"
password = "Baodai123!"

common = pypi_xmlrpc.ServerProxy('%s/xmlrpc/2/common' % server)
print(common.version())
odooApi = pypi_xmlrpc.ServerProxy('%s/xmlrpc/2/object' % server)
odooApi
uid = common.authenticate(database,user,password,{})
uid

{'server_version': '17.0-20241209', 'server_version_info': [17, 0, 0, 'final', 0, ''], 'server_serie': '17.0', 'protocol_version': 1}


2

# Lấy danh sách cửa hàng

In [5]:
# query = """
# SELECT * 
# FROM shop@summary 
# WHERE shop_type=3 
# AND center_code='5'
# AND shop_status =1
# AND province_code = 'BKA'
# """
# AND province_code = 'BNI'
# AND shop_code = '5BNI00001111'
# AND (shop_name LIKE '%Cửa hàng%' OR shop_name LIKE '%CH%' OR shop_name LIKE '%Trung tâm giao dịch%' OR shop_name LIKE '%TTGD%')
shop_code_list=(
'5TNG00001',
'5BKA00002'
)

query = f"""
SELECT * 
FROM shop@summary 
WHERE shop_code in {shop_code_list}
"""
# AND parent_shop_code ='CT5'
# """
df_shop = pd.read_sql(query, engine)
df_shop

,shop_id,shop_code,shop_name,shop_type,province_code,parent_shop_id,billing_shop_code,shop_status,center_code,shop_address,parent_shop_code,price_policy,discount_policy,check_stock,tel_number,tax_code,account,pay_comm,agent_type,sun_shop_code,shop_accountant,collection_group_id,issue_date,issue_place,id_no,company_name,contact_name,cashier,comm_ser_rate,comm_ser_rate_type,use_invoice,sta_date,end_date,sun_shop_accountant,agent_charge_receipt,collection_group_transfer_id,comm_pay_rate_type,comm_pay_rate,old_shop_code,sun_t0_code,sun_stock_code,sun_payment_code,direct_sale,sun_shop_name,comm_shop_type,alert_mail,district_code,distribution_channel_type,distribution_channel_code,area_code,old_center_code,old_parent_shop_id,old_parent_shop_code,alert_cash_amount,alert_transfer_amount,shop_online,lat,lng,last_update
0,4258,5BKA00002,Cửa Hàng Bắc Cạn,3,BKA,38556,5BKA00002,1,5,"Tổ 7, Phường Đức Xuân, Thành phố Bắc Kạn, tỉnh...",5KVCBBC2,1,1,1,None,0100686209-060,None,1,3,07102,None,1600105,None,None,None,None,None,None,3,1,2,None,None,None,0,None,1,None,None,None,None,None,0,None,1,None,None,1,None,None,5,6059,5KVCBBC,None,None,None,None,None,2022-03-30 17:22:09
1,4261,5TNG00001,Cửa Hàng Thái Nguyên,3,TNG,4281,5TNG00001,1,5,"135A, Đường Cách mạng tháng Tám Thành phố Thái...",5CNHTT,1,1,1,None,0100686209-092,None,1,3,10101,Nguyễn Thị Thu Huyền,1600095,None,None,None,None,Nguyễn Thị Thu Huyền,None,3,1,2,None,None,None,0,None,1,None,None,None,None,None,0,None,1,None,None,1,None,None,5,4281,5CNHTT,None,None,None,None,None,2022-03-30 08:56:06


In [6]:
df_shop['province_code'].unique()

array(['HDU'], dtype=object)

# Tạo tỉnh/ Thành Phố

In [6]:
province_dict = {
    "QNI": "Quảng Ninh",
    "HDU": "Hải Dương",
    "HYE": "Hưng Yên",
    "BNI": "Bắc Ninh",
    "BGI": "Bắc Giang",
    "TNG": "Thái Nguyên",
    "CBA": "Cao Bằng",
    "BKA": "Bắc Kạn",
    "LSO": "Lạng Sơn",
    "TBI": "Thái Bình"
}
# department_dict = [
#     {'name': "Quảng Ninh"},
#     {'name': "Hải Dương"},
#     {'name': "Hưng Yên"},
#     {'name': "Bắc Ninh"},
#     {'name': "Bắc Giang"},
#     {'name': "Thái Nguyên"},
#     {'name': "Cao Bằng"},
#     {'name': "Bắc Kạn"},
#     {'name': "Lạng Sơn"},
# ]
# create_result = odooApi.execute_kw(database,uid,password,'hr.department','create',[department_dict])
# print("create result",create_result)

In [7]:
department_data = odooApi.execute_kw(database,uid,password,'hr.department','search_read',[],{"fields":["id","name"]})
department_dict = {item['name']: item['id'] for item in department_data}
department_dict

{'AM Bắc Giang': 131,
 'AM Bắc Kạn': 134,
 'AM Bắc Ninh': 130,
 'AM Cao Bằng': 133,
 'AM Hưng Yên': 129,
 'AM Hải Dương': 128,
 'AM Hải Phòng': 126,
 'AM Lạng Sơn': 135,
 'AM Quảng Ninh': 127,
 'AM Thái Bình': 136,
 'AM Thái Nguyên': 132,
 'Administration': 1,
 'Bắc Cạn_Trung tâm giao dịch huyện Chợ Mới': 65,
 'Bắc Giang': 29,
 'Bắc Kạn': 32,
 'Bắc Ninh': 28,
 'CH 146 Trần Phú': 108,
 'CH 4T - Hải Phòng': 6,
 'CH Bắc Giang': 38,
 'CH Cao Bằng': 36,
 'CH Hạ Long Quảng Ninh': 39,
 'CH Lạng Sơn': 37,
 'CH cẩm phả': 35,
 'Cao Bằng': 31,
 'Cửa Hàng Bắc Ninh': 138,
 'Cửa Hàng Hải Dương': 124,
 'Cửa Hàng Móng Cái': 137,
 'Cửa Hàng Thái Bình': 123,
 'Cửa hàng 231A Lạch Tray': 3,
 'Cửa hàng 380 Tô Hiệu': 7,
 'Cửa hàng Hưng Yên': 34,
 'Cửa hàng Hồng Hà': 106,
 'Cửa hàng Uông Bí': 40,
 'Hưng Yên': 27,
 'Hải Dương': 26,
 'Hải Phòng': 2,
 'Lạng Sơn': 33,
 'Phòng KD dịch vụ viễn thông': 120,
 'Phòng Tổng hợp': 119,
 'Phòng chăm sóc khách hàng': 122,
 'Phòng kế toán': 121,
 'Quảng Ninh': 25,
 'TTGD H

# Tạo các cửa hàng

In [8]:
shop_data_list = []
for shop in df_shop.to_dict('records'):
    shop_code = shop['shop_code']
    shop_name = shop['shop_name']
    province_code = shop['province_code']
    province_id = department_dict[province_dict[province_code]]
    shop_data = {
        "name": shop_name,
        "parent_id": province_id,
        "shop_code": shop_code
    }
    shop_data_list.append(shop_data)
# print(shop_data_list)
create_result = odooApi.execute_kw(database,uid,password,'hr.department','create',[shop_data_list])
create_result

[139, 140]

In [9]:
department_data = odooApi.execute_kw(database,uid,password,'hr.department','search_read',[],{"fields":["id","name","shop_code"]})
department_dict = {item['shop_code']: item['id'] for item in department_data}
print(len(department_dict))
department_dict

5


{False: 4,
 '5BKA00002': 139,
 '5BNI00001': 138,
 '5QNI00011': 137,
 '5TNG00001': 140}

# Tạo nhân viên

In [10]:
shop_code_list_str =  str(df_shop['shop_code'].to_list())
shop_code_list_str = shop_code_list_str.replace('[','(')
shop_code_list_str = shop_code_list_str.replace(']',')')
query = f"""
SELECT * 
FROM employee@summary
WHERE status = 1
AND shop_code in {shop_code_list_str}
"""
# print(query)
df_employee = pd.read_sql(query, engine)
df_employee = df_employee[['emp_code','emp_name','shop_code']]
df_employee

,emp_code,emp_name,shop_code
0,OANHPTK,Phí Thị Kiều Oanh,5TNG00001
1,C1_HUYENNT,Nguyễn Thị Thu Huyền,5TNG00001
2,HONGTT,Trần Thu Hồng ( Đón Tiếp ),5TNG00001
3,HIEUDQ,Đoàn Quang Hiếu,5TNG00001
4,HUEVT,Vũ Thị Huệ,5TNG00001
5,C5_PHUONGTT,Trần Thị Phương,5TNG00001
6,C5_HOAHT,Hoàng Thị Hòa,5TNG00001
7,C5_HUONGHT,Hoàng Thị Thương,5TNG00001
8,C5_TNG_TP_TT,Thái Nguyên_Cửa Hàng Thái Nguyên,5TNG00001
9,C5_THAONTP2,Nguyễn Thị Phương Thảo,5BKA00002


In [11]:
def get_department_id(shop_code):
    return department_dict[shop_code]
df_employee['department_id'] = df_employee['shop_code'].apply(get_department_id)
df_employee = df_employee[['emp_name','emp_code','department_id']]
df_employee.rename(columns={'emp_code': 'emp_code_mobifone', 'emp_name': 'name'}, inplace=True)
df_employee_dict = df_employee.to_dict('records')
create_result = odooApi.execute_kw(database,uid,password,'hr.employee','create',[df_employee_dict])

C:\Users\admin\AppData\Local\Temp\ipykernel_16916\410217847.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_employee.rename(columns={'emp_code': 'emp_code_mobifone', 'emp_name': 'name'}, inplace=True)


In [19]:
create_result

[1002,
 1003,
 1004,
 1005,
 1006,
 1007,
 1008,
 1009,
 1010,
 1011,
 1012,
 1013,
 1014,
 1015,
 1016,
 1017,
 1018,
 1019,
 1020,
 1021,
 1022,
 1023,
 1024,
 1025,
 1026,
 1027,
 1028,
 1029,
 1030,
 1031,
 1032,
 1033]

In [26]:
shop_code_list_str[0]

'['

# Xóa cửa hàng không có nhân viên

In [8]:
# department_list = odooApi.execute_kw(database,uid,password,'hr.department','search',[[('member_ids','=',False)]],{"limit":0})
# odooApi.execute_kw(database,uid,password,'hr.department','unlink',[department_list])

In [1]:
import datetime

x = datetime.datetime.now()
print(x)

2025-02-10 10:53:13.216767


In [4]:
x.hour

15

In [3]:
x

datetime.datetime(2025, 2, 7, 15, 54, 35, 54235)

In [3]:
import datetime
t = datetime.datetime(2012, 2, 23, 0, 0)
t.strftime('%m/%d/%Y')

'02/23/2012'

In [11]:
val = {
    'employee_id': 213,
    'check_in': datetime.datetime.now(),

}
query = (f'''
    SELECT MIN(id) FROM hr_attendance
    WHERE check_in::DATE =  {val['check_in'].strftime('%m-%d-%Y')}
    AND employee_id = {val['employee_id']};
''')
query

'\n    SELECT MIN(id) FROM hr_attendance\n    WHERE check_in::DATE =  02-10-2025\n    AND employee_id = 213;\n'

In [9]:
print(query)

('\n    SELECT MIN(id) FROM hr_attendance\n    WHERE check_in::DATE =  %s\n    AND employee_id = %s;\n', '02-10-2025', 213)
